# CRIAÇÃO DA CLASSE DE EXTRAÇÃO DE FEATURES

Esse jupyter tem como objetivo implementar o maior número possível de funções para extração de features. Essas funções serão testadas para que depois possa ser criada uma classe.

In [1]:
import os
import librosa
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from IPython.display import Audio

## Tamanho do janelamento

É preciso analisar o dataset para verificar qual vai ser o tamanho do janelamento. Para garantir que o processamento seja o mais rápido possível, o tamanho da janela será o tamanho do menor áudio.

As células abaixo verificarão qual é o tamanho do menor arquivo do dataset.

In [2]:
arrayDuracoes = []

diretorio = "/home/dimi/Downloads/SESA/test/"
for arquivo in os.listdir(diretorio):
    arrayDuracoes.append(librosa.get_duration(filename=diretorio + arquivo))

diretorio = "/home/dimi/Downloads/SESA/train/"
for arquivo in os.listdir(diretorio):
    arrayDuracoes.append(librosa.get_duration(filename=diretorio + arquivo))

In [3]:
print("Duração mínima:", min(arrayDuracoes))
print("Duração máxima:", max(arrayDuracoes))

Duração mínima: 1.0715625
Duração máxima: 33.030375


A célula abaixo vai setar o valor da variável do janelamento. FRAME_TIME e OVERLAP_TIME estarão em segundos, já FRAME_LENGTH e OVERLAP_LENGTH estarão em amostras.

In [4]:
audioTesteDir = "/home/dimi/Downloads/SESA/train/casual_042.wav" 
audioTeste, freqAmostragem = librosa.load(audioTesteDir, sr=None, mono=True)

frameTime     = min(arrayDuracoes)
overlapTime   = frameTime / 2

frameLength   = int(freqAmostragem * frameTime)
overlapLength = int(freqAmostragem * overlapTime)

print("Frequência de amostragem do dataset:\t", freqAmostragem)
print("Tamanho do janelamento (segundos):\t", frameTime)
print("Tamanho do janelamento (amostras):\t", frameLength)

Frequência de amostragem do dataset:	 16000
Tamanho do janelamento (segundos):	 1.0715625
Tamanho do janelamento (amostras):	 17145


## Criando e testando as funções que vão extrair as features

Para testar as funções, vou utilizar um áudio qualquer, que já foi definido na célula anterior.

In [5]:
print("Tamanho do áudio teste (amostras):\t", len(audioTeste))
print("Tamanho do janelamento (amostras):\t", frameLength)

# PARA SABER O NÚMERO DE JANELAS, TENHO QUE MULTIPLICAR POR 2 ALÍ PQ AINDA TEM A SOBREPOSIÇÃO
# E SOMAR UM PQ SEMPRE VAI SOBRAR UM TECO
print("\nQtd de janelas do áudio de teste:\t", 1 + 2*int(len(audioTeste)/frameLength))

Tamanho do áudio teste (amostras):	 196000
Tamanho do janelamento (amostras):	 17145

Qtd de janelas do áudio de teste:	 23


### Features unitárias por janela

Primeiro, vou criar as funções para extrair as features que retornam somente um valor para cada janela do áudio. Vou deixar MFCCs, Chromas e Contrastes pra depois.

#### RMS

In [6]:
def extrairRMS(sinal, frameLength, overlapLength):
    return librosa.feature.rms(y=sinal, frame_length=frameLength, hop_length=overlapLength)

In [7]:
rms = extrairRMS(audioTeste, frameLength, overlapLength)
print("Total:", len(rms[0]))
print(rms)

Total: 23
[[0.97289884 0.97283065 0.9683342  0.95529735 0.9353164  0.9231679
  0.9189752  0.91432136 0.91490877 0.9199677  0.9292392  0.93753517
  0.9363619  0.9375422  0.94477415 0.9487146  0.932465   0.91362005
  0.9123069  0.9129697  0.92793566 0.94241256 0.9497184 ]]


#### Centróide Espectral

In [8]:
def extrairCentroideEspectral(sinal, freqAmostragem, frameLength, overlapLength):
    return librosa.feature.spectral_centroid(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

In [9]:
centroideEspectral = extrairCentroideEspectral(audioTeste, freqAmostragem, frameLength, overlapLength)
print("Total:", len(centroideEspectral[0]))
print(centroideEspectral)

Total: 23
[[3377.06116864 3405.16455392 3408.64093306 3153.77782842 2952.23849722
  3297.5263438  3498.19650293 3555.64722383 3510.93774387 3328.77180695
  3453.71077273 3464.53747956 3377.10366306 3371.66623924 3356.21156993
  3365.48349049 3214.24186212 3326.15057732 3484.24147863 3410.43198092
  3389.76753095 3088.52522657 3059.16436228]]


#### Largura de banda espectral

In [10]:
def extrairLarguraBanda(sinal, freqAmostragem, frameLength, overlapLength):
    return librosa.feature.spectral_bandwidth(y=sinal, sr= freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

In [11]:
larguraBanda = extrairLarguraBanda(audioTeste, freqAmostragem, frameLength, overlapLength)
print("Total:", len(larguraBanda[0]))
print(larguraBanda)

Total: 23
[[2303.6356116  2274.54695736 2291.14823972 2286.43795123 2302.79484
  2270.56544052 2237.08173489 2212.32414584 2229.74843455 2280.92561942
  2237.15157496 2216.4406777  2245.91932893 2233.38044745 2249.29224195
  2256.34467685 2267.90052923 2273.34275505 2241.7598426  2278.28100339
  2260.01232179 2289.73748255 2335.49885123]]


#### Planicidade espectral

In [12]:
def extrairPlanicidade(sinal, frameLength, overlapLength):
    return librosa.feature.spectral_flatness(y=sinal, n_fft=frameLength, hop_length=overlapLength)

In [13]:
planicidade = extrairPlanicidade(audioTeste, frameLength, overlapLength)
print("Total:", len(planicidade[0]))
print(planicidade)

Total: 23
[[0.20862587 0.42513445 0.4082259  0.3744419  0.31562665 0.41912323
  0.43904808 0.48385513 0.46347493 0.42050347 0.4376149  0.44379652
  0.43769926 0.4510455  0.44393855 0.45216855 0.36779508 0.3998395
  0.4537909  0.43156722 0.43557653 0.36260265 0.34631407]]


#### Rolloff espectral

In [14]:
def extrairRolloff(sinal, freqAmostragem, frameLength, overlapLength):
    return librosa.feature.spectral_rolloff(y=sinal, sr= freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

In [15]:
rolloff = extrairRolloff(audioTeste, freqAmostragem, frameLength, overlapLength)
print("Total:", len(rolloff[0]))
print(rolloff)

Total: 23
[[6262.24918339 6242.65048997 6232.3845077  6079.3280448  5940.27064862
  6163.32244517 6276.24825012 6283.71441904 6268.78208119 6211.85254316
  6270.64862343 6249.18338777 6257.58282781 6183.85440971 6216.51889874
  6198.78674755 6071.86187587 6181.05459636 6256.6495567  6284.64769015
  6211.85254316 5980.40130658 6044.79701353]]


#### Taxa de cruzamentos por zero

In [16]:
def extrairZCR(sinal, frameLength, overlapLength):
    return librosa.feature.zero_crossing_rate(y=sinal, frame_length=frameLength, hop_length=overlapLength)

In [17]:
zcr = extrairZCR(audioTeste, frameLength, overlapLength)
print("Total:", len(zcr[0]))
print(zcr)

Total: 23
[[0.16809565 0.33105862 0.31904345 0.261359   0.2208224  0.26625838
  0.3133275  0.32907553 0.32621756 0.30492855 0.3074949  0.31659376
  0.3007874  0.29291339 0.30072908 0.28626422 0.24596092 0.2671916
  0.29827938 0.30335375 0.28632254 0.22315544 0.19492563]]


### Features não unitárias por janela

Agora sim é hora de extrair os MFCCs, Chromas e Contrastes.

#### MFCC

A função do MFCC retorna uma matriz, e ela será necessária para calcular os deltas e delta deltas, mas, na verdade, o MFCC que será colocado no array de features é uma média de cada linha da matriz. Por isso, primeiro vamos criar uma matriz do MFCC, e só depois a função extrairMediaMFCC vai fazer a média.

In [18]:
def extrairMatrizMFCC(sinal, freqAmostragem):
    return librosa.feature.mfcc(y=sinal, sr=freqAmostragem)

In [19]:
matrizMFCC = extrairMatrizMFCC(audioTeste, freqAmostragem)

print("Qtd linhas (MFCCs):", len(matrizMFCC))
print("Qtd de valores em cada linha:", len(matrizMFCC[0]))

Qtd linhas (MFCCs): 20
Qtd de valores em cada linha: 383


In [20]:
def extrairMFCCs(matrizMFCC):
    
    arrayMFCCs = []
    
    for linha in matrizMFCC:
        arrayMFCCs.append(np.mean(linha))
        
    return arrayMFCCs

In [21]:
MFCCs = extrairMFCCs(matrizMFCC)

print("Qtd de MFCCs:", len(MFCCs))
print(MFCCs)

Qtd de MFCCs: 20
[218.63036349922476, 21.060018131195708, -5.090762600082968, 2.269630059832939, -9.334851730565067, -0.9481667939149704, -3.606366439455067, -6.228105011358649, -2.425939278288877, -1.3700105930352873, -4.620659252854622, -1.8810377388475406, 1.0335270075937133, -1.6101901035430177, 1.4737038717078546, 1.4784275279571726, 0.12237327841078621, 2.5565200359192013, -0.7213633653903515, -1.0908674502864897]


#### MFCC Delta

Assim como o MFCC, a função do librosa que extrai os deltas retorna uma matriz. A função abaixo retorna a média de cada linha dessa matriz.

In [22]:
def extrairDeltas(matrizMFCC):
    matrizDelta = librosa.feature.delta(matrizMFCC, order=1)

    arrayDelta = []

    for linha in matrizDelta:
        arrayDelta.append(np.mean(linha))

    return arrayDelta

In [23]:
deltas = extrairDeltas(matrizMFCC)

print("Qtd de deltas:", len(deltas))
print(deltas)

Qtd de deltas: 20
[0.00845555149101118, 0.035612215409566196, 0.002806821564038323, -0.002518456302930467, 0.009241424540702784, -0.0034264949423760413, -0.0015540583280829847, -0.0032644520250503365, -0.0035893956286237464, 0.007832486321244192, 0.016569689768331678, 0.0022751168282840093, 0.009231983347610237, 0.023039515224229916, -0.007992250274650295, 0.0009103221229303783, 0.030793369813422494, 0.019662701818535067, 0.012580190237291808, -0.0042172102548367835]


#### MFCC Delta Delta

In [24]:
def extrairDeltaDeltas(matrizMFCC):
    matrizDeltaDelta = librosa.feature.delta(matrizMFCC, order=2)

    arrayDeltaDelta = []

    for linha in matrizDeltaDelta:
        arrayDeltaDelta.append(np.mean(linha))

    return arrayDeltaDelta

In [25]:
deltaDeltas = extrairDeltaDeltas(matrizMFCC)

print("Qtd de delta deltas:", len(deltaDeltas))
print(deltaDeltas)

Qtd de delta deltas: 20
[-0.010601507118590004, -0.0010596455903566708, -0.008728935271422093, 0.007129450561897048, 0.004319593045042994, -0.012841760312570177, -0.016290592470991832, -0.006092034699096264, -0.006237431084616262, -0.007912498869041532, -0.005202688361176131, -0.00901883527713022, 0.002189074129611164, 0.003987256232567145, 0.005828932534320377, 0.0024293182557689535, 0.002831223024423794, 0.005637318042738157, 0.004555332952872064, 0.0026232203083446206]


#### Mel Espectrograma

Essa também retorna uma matriz, mas ela vem invertida, tenho que tirar a média de cada COLUNA.

In [26]:
def extrairMelEspectrograma(sinal, freqAmostragem, frameLength, overlapLength):
    
    matrizMelEspectrograma = librosa.feature.melspectrogram(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)
    
    arrayMelEspectrograma = []
    
    for coluna in matrizMelEspectrograma.T:
        arrayMelEspectrograma.append(np.mean(coluna))
    
    return arrayMelEspectrograma

In [27]:
melEspectrograma = extrairMelEspectrograma(audioTeste, freqAmostragem, frameLength, overlapLength)

print("Qtd de features:", len(melEspectrograma))
print(melEspectrograma)

Qtd de features: 23
[9570.04256992491, 8755.53390849765, 8899.649301026251, 10259.089626841838, 11136.780923949173, 9016.744580308401, 7690.262057484933, 7267.0870558676415, 7503.434648866611, 8777.807106707889, 7956.240701175975, 7851.911342069301, 8407.602126082898, 8477.264668694304, 8788.676597168058, 8913.930068147038, 9871.661309884461, 9113.27421887679, 7845.123072620453, 8205.829810018346, 8403.363909387137, 10903.044633953104, 11445.250868945495]


#### Cromagrama

Essa também retorna uma matriz. Ela tem 12 cromagramas (linhas). Vou tirar a média de cada linha.

In [28]:
def extrairCromagramas(sinal, freqAmostragem, frameLength, overlapLength):
    
    matrizCromagramas = librosa.feature.chroma_stft(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)
    
    arrayCromagramas = []
    
    for linha in matrizCromagramas:
        arrayCromagramas.append(np.mean(linha))
    
    return arrayCromagramas

In [29]:
cromagramas = extrairCromagramas(audioTeste, freqAmostragem, frameLength, overlapLength)
print("Features:", len(cromagramas))
print(cromagramas)

Features: 12
[0.6008743810470407, 0.6192316961822517, 0.6667679860367414, 0.5825510775417423, 0.593618639982298, 0.7165458847216384, 0.8398419559414932, 0.8229532372995757, 0.7429803219273489, 0.7372121606363919, 0.6538187863614455, 0.568055629925344]


#### Cromagrama de constante Q

Essa função não precisa do tamanho da janela (frameLength) e a sobreposição: "hop_length must be a positive integer multiple of 2^6 for 7-octave CQT". Portanto, vou deixar esse parâmetro como padrão.

In [30]:
def extrairCromagramasQ(sinal, freqAmostragem):
    
    matrizCromagramasQ = librosa.feature.chroma_cqt(y=sinal, sr=freqAmostragem)
    
    arrayCromagramasQ = []
    
    for linha in matrizCromagramasQ:
        arrayCromagramasQ.append(np.mean(linha))
    
    return arrayCromagramasQ

In [31]:
cromagramasQ = extrairCromagramasQ(audioTeste, freqAmostragem)
print("Features:", len(cromagramasQ))
print(cromagramasQ)

Features: 12
[0.7423326366339749, 0.8118458760732015, 0.7749866599976304, 0.6888570958887758, 0.6844563707706994, 0.720939679150639, 0.7696370006528446, 0.772528034411687, 0.7429615093198946, 0.7324581398205714, 0.708322482131199, 0.6633871173325862]


#### Croma CENS

Assim como o cromagrama de constante Q, essa função não precisa do tamanho da janela (frameLength) e a sobreposição: "hop_length must be a positive integer multiple of 2^6 for 7-octave CQT". Portanto, vou deixar esse parâmetro como padrão.

In [32]:
def extrairCromaCENSs(sinal, freqAmostragem):
    
    matrizCromaCENSs = librosa.feature.chroma_cens(y=sinal, sr=freqAmostragem)
    
    arrayCromaCENSs = []
    
    for linha in matrizCromaCENSs:
        arrayCromaCENSs.append(np.mean(linha))
    
    return arrayCromaCENSs

In [33]:
cromaCENSs = extrairCromaCENSs(audioTeste, freqAmostragem)
print("Features:", len(cromaCENSs))
print(cromaCENSs)

Features: 12
[0.2889321013998348, 0.33092915177628063, 0.3030257216642103, 0.25846359087921816, 0.2620416379149073, 0.27861773447656585, 0.2986214485837028, 0.3100619694287311, 0.2937169209459597, 0.28334975869294, 0.2724221661114414, 0.25863450842997304]


#### Contraste espectral

As ultimas features serão essas.

In [34]:
def extrairContrastes(sinal, freqAmostragem, frameLength, overlapLength):
    
    matrizContrastes = librosa.feature.spectral_contrast(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)
    
    arrayConstrastes = []
    
    for linha in matrizContrastes:
        arrayConstrastes.append(np.mean(linha))
    
    return arrayConstrastes

In [35]:
contrastes = extrairContrastes(audioTeste, freqAmostragem, frameLength, overlapLength)
print(contrastes)

[16.726175997469667, 14.92242325103805, 15.14579009848466, 15.0678114625672, 15.051812102215814, 15.290737977844193, 14.06930378888526]


## Criando a classe

O objetivo é que essa classe entre numa pasta e me devolva uma matriz em que cada linha seja um áudio e cada coluna seja uma feature. Essa classe ainda deve ser capaz de gravar tudo isso num csv.

As features serão extraídas, e depois que tudo já estiver pronto, ela vai escalonar as features e depois passar tudo num PCA.

Então, para usar a classe, é necessário ter as pastas de áudios muito bem organizadas para que ela crie o CSV dentro da pasta que se queira. Preferi fazer assim, pois, se eu fosse abrir todos os áudios numa matriz do python pra só depois mandar pra essa classe, poderia ficar mto pesado. É melhor que ela só receba o caminho pra uma pasta e gere um CSV.

### Criando funções intermediárias

Vou usar essa parte pra criar as funções intermediárias, que não serão responsáveis por calcular nenhuma feature, mas apenas montar os arrays e metrizes da classe.

#### Função que faz o janelamento

Existe um problema em deixar que as funções de extração de features criadas acima façam o janelamento: ao invés de retornarem valores unitários para as features, elas vão retornar um array em que cada posição representa um janelamento. Portanto, a solução é fazer o janelamento antes de extrair as features e deixar para mandar para essas funções apenas as janelas, fazendo com que frameLength seja igual ao tamanho da janela que está sendo enviada e que overlapLength seja 0.

A função abaixo usa a função frame do librosa que retorna as janelas como COLUNAS. Como eu quero que cada janela seja uma LINHA, eu retorno a transposta dessa função.

In [36]:
def fazerJanelamento(sinal, frameLength, overlapLength):
    return librosa.util.frame(sinal, frame_length=frameLength, hop_length=overlapLength).T

In [37]:
framesAudioTeste = fazerJanelamento(audioTeste, frameLength, overlapLength)
print("Qtd janelas:", len(framesAudioTeste))
print(framesAudioTeste)
print(framesAudioTeste[0])

Qtd janelas: 21
[[ 0.9609375  0.9375     0.96875   ...  0.9375     0.921875   0.953125 ]
 [ 0.9609375  0.9765625 -1.        ...  0.828125   0.8671875  0.8984375]
 [ 0.953125   0.9609375  0.984375  ...  0.8515625  0.9140625 -1.       ]
 ...
 [-0.9765625  0.9921875  0.8671875 ...  0.9453125  0.90625    0.9453125]
 [-0.96875   -0.96875    0.984375  ...  0.9765625  0.9765625  0.9765625]
 [ 0.9453125  0.9375    -0.9609375 ...  0.921875   0.8828125  0.8515625]]
[0.9609375 0.9375    0.96875   ... 0.9375    0.921875  0.953125 ]


In [38]:
Audio(data=framesAudioTeste[0], rate=freqAmostragem)

#### Função que extrai features de um único frame

Em algum momento, a classe deverá passar em cada um dos áudios da pasta para ir extraindo as features. A próxima função extrai as features de um único frame de áudio e retorna um array com essas features. Posteriormente esse array deverá ser integrado à matriz de features de todos os áudios.

Haverá uma outra função para extrair as features de um único áudio. Ela deverá pegar um áudio, usar a função de janelamento, e usar a função abaixo para extrair as features de cada uma das janelas. Depois, ela vai retornar uma matriz com as features de cada frame de um único áudio.

PARA IMPEDIR QUE O LIBROSA CONTINUE FAZENDO O JANELAMENTO DO ÁUDIO MESMO QUE frameLength SEJA DO TAMANHO DO ÁUDIO, O PARÂMETRO DE OVERLAP DEVE SER MAIOR OU IGUAL A frameLength.

In [39]:
def extrairFeaturesUnicoFrame(sinal, freqAmostragem, frameLength):
    
    # PARA IMPEDIR QUE O LIBROSA CONTINUE FAZENDO O JANELAMENTO DO ÁUDIO MESMO QUE frameLength 
    # SEJA DO TAMANHO DO ÁUDIO, O PARÂMETRO DE OVERLAP DEVE SER MAIOR OU IGUAL A frameLength:
    overlapLength = frameLength
    
    # CRIANDO O ARRAY DE FEATURES DO FRAME EM QUESTAO
    arrayFeaturesFrame = []
    
    #PRIMEIRO, VOU EXTRAIR AS FEATURES UNITARIAS
    arrayFeaturesFrame.append(float(extrairRMS(sinal, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairCentroideEspectral(sinal, freqAmostragem, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairLarguraBanda(sinal, freqAmostragem, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairPlanicidade(sinal, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairRolloff(sinal, freqAmostragem, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairZCR(sinal, frameLength, overlapLength)))
    
    # AGORA VAMOS PASSAR PARA AS NAO UNITARIAS, PRIMEIRO, E PRECISO CRIAR A MATRIZ DOS MFCCS
    matrizMFCC          = extrairMatrizMFCC(sinal, freqAmostragem)
    
    # AGORA SIM EU SAIO EXTRAINDO AS FEATURES 
    arrayFeaturesFrame += extrairMFCCs(matrizMFCC)
    arrayFeaturesFrame += extrairDeltas(matrizMFCC)
    arrayFeaturesFrame += extrairDeltaDeltas(matrizMFCC)
    arrayFeaturesFrame += extrairMelEspectrograma(sinal, freqAmostragem, frameLength, overlapLength)
    arrayFeaturesFrame += extrairCromagramas(sinal, freqAmostragem, frameLength, overlapLength)
    arrayFeaturesFrame += extrairCromagramasQ(sinal, freqAmostragem)
    arrayFeaturesFrame += extrairCromaCENSs(sinal, freqAmostragem)
    arrayFeaturesFrame += extrairContrastes(sinal, freqAmostragem, frameLength, overlapLength)
    
    # POR FIM, RETORNO O ARRAY DE FEATURES DO AUDIO QUE FOI ENVIADO PARA ESSA FUNCAO
    return arrayFeaturesFrame

In [40]:
frameAtual = framesAudioTeste[0]

featuresFrameAtual = extrairFeaturesUnicoFrame(frameAtual, freqAmostragem, frameLength)

print(len(featuresFrameAtual))
print(featuresFrameAtual)

110
[0.9728988409042358, 3377.061168638039, 2303.6356116043476, 0.20862586796283722, 6262.249183387775, 0.16809565470982793, 221.53280706804168, 21.19834589785677, 1.0012264928341856, 5.263567737867123, -8.234803250169637, 2.916007312866955, -1.7485395399910024, -3.9926662851803836, -0.7837956354089776, -1.28079885661594, -0.9368607871097065, -0.6277930498722697, 1.5727513515461322, -3.1451725351384363, 0.18228527875054762, -0.4002125886253895, -1.5030519704526164, 0.20320440111907276, -3.5990314363883735, 1.9381680950506794, 0.08905400207585759, -0.098222011994975, -0.054339009052107354, 0.09565601439595162, 0.14975389468396536, 0.08222962635775, -0.0108842410290983, -0.16757563495637334, -0.10258308837954433, 0.12241220169860786, -0.01720994072219128, -0.09928815736885213, 0.1588778357067479, 0.10337051105258759, -0.32752447318415245, -0.09067838123138729, 0.06829057133578621, -0.21333502680294134, -0.11051394327897336, -0.050314484934365286, -0.029874794161612955, -0.105253071992671

#### Função que extrai as features de um único áudio

Essa função vai pegar um áudio, usar a função de janelamento, e para cada janela do áudio em questão, ela vai usar a função de extrair as features de uma única janela (implementada acima). Depois, ela vai retornar uma matriz com as features de cada frame de um único áudio, onde cada linha é uma frame e cada coluna é uma feature.

In [41]:
def extrairFeaturesUnicoAudio(sinal, freqAmostragem, frameLength, overlapLength):
    
    # PRIMEIRO, VOU CRIAR A MATRIZ QUE VAI CONTER AS FEATURES DE CADA JANELA
    # CADA LINHA E UMA JANELA E CADA COLUNA E UMA FEATURE
    matrizFeaturesAudio = []
    
    # DEPOIS, VOU FAZER O JANELAMENTO
    matrizFramesAudio = fazerJanelamento(sinal, frameLength, overlapLength)
    
    # AGORA, PARA CADA JANELA, VOU EXTRAIR AS FEATURES E COLOCAR COMO UMA LINHA NOVA NA MATRIZ
    for frameAtual in matrizFramesAudio:
        matrizFeaturesAudio.append(extrairFeaturesUnicoFrame(frameAtual, freqAmostragem, frameLength))
    
    # RETORNO A MATRIZ DE FEATURES DESSE AUDIO
    return matrizFeaturesAudio

In [42]:
matrizFeaturesAudioTeste = extrairFeaturesUnicoAudio(audioTeste, freqAmostragem, frameLength, overlapLength)
print("Qtd frames (linhas):", len(matrizFeaturesAudioTeste))
print("Qtd features (colunas):", len(matrizFeaturesAudioTeste[0]))

Qtd frames (linhas): 21
Qtd features (colunas): 110


#### Função que coloca o nome do arquivo e a classificação correta na matriz de features de um áudio

Haverá uma matriz de dados que terá as seguintes colunas: **nomeArquivo, ...features... e classificacaoCorreta**. Mas, a função implementada para gerar a matriz de features de um único áudio **extrairFeaturesUnicoAudio** devolve uma matriz de features **sem** o nome do áudio e a classificação correta. Portanto, a função abaixo apenas pega essa última matriz e coloca o nome do arquivo no começo e a classificação correta ao final. Posteriormente, o resultado será agregado à matriz de dados citada em primeiro lugar.

ESSA FUNÇÃO RETORNA UM DATAFRAME PANDAS, NÃO UMA MATRIZ QUALQUER.

https://www.geeksforgeeks.org/python-pandas-dataframe-insert/

In [43]:
def adicionarNomeArquivoEClasse(matrizFeaturesAudioAtual, nomeArquivo, classificacaoCorreta):
    # PRIMEIRO TRANSFORMO A MATRIZ NUM PANDAS DATAFRAME
    dataframeAudioAtual = pd.DataFrame(matrizFeaturesAudioAtual)
    
    # AGORA COLOCO A COLUNA DO NOME NO COMECO (posicaoNovaColuna, nomeNovaColuna, valorParaTodasAsLinhas)
    dataframeAudioAtual.insert(0, "nomeArquivo", nomeArquivo, True)
    
    # AGORA COLOCO A COLUNA DA CLASSIFICACAO CORRETA NA ULTIMA POSICAO
    dataframeAudioAtual.insert(len(dataframeAudioAtual.columns), "classificacaoCorreta", classificacaoCorreta, True)
    
    return dataframeAudioAtual

In [44]:
dataframeAudioTeste = adicionarNomeArquivoEClasse(matrizFeaturesAudioTeste, "teste.wav", "gunshot")
dataframeAudioTeste

,nomeArquivo,0,1,2,3,4,5,6,7,8,...,101,102,103,104,105,106,107,108,109,classificacaoCorreta
0,teste.wav,0.972899,3377.061169,2303.635612,0.208626,6262.249183,0.168096,221.532807,21.198346,1.001226,...,0.243921,0.270332,23.005447,23.314442,23.074566,22.576420,23.924732,24.934543,24.205152,gunshot
1,teste.wav,0.972763,3339.798126,2279.350128,0.215444,6198.786748,0.162963,221.557982,22.959761,0.059361,...,0.233492,0.260226,24.615355,24.632467,20.934591,21.107852,23.117511,25.457013,24.708157,gunshot
2,teste.wav,0.963885,3418.727827,2271.968297,0.197985,6202.519832,0.156080,220.457246,30.920611,-0.789945,...,0.247476,0.259320,29.820945,22.032529,27.410318,23.200435,23.896292,26.190899,23.940436,gunshot
3,teste.wav,0.946629,3126.108572,2312.871959,0.184629,6085.860943,0.105279,219.212834,34.936589,-2.596156,...,0.258251,0.258423,29.500904,22.654341,22.274284,25.839259,24.424743,23.936114,22.598811,gunshot
4,teste.wav,0.923872,2975.281584,2297.815918,0.159496,5959.869342,0.115544,218.023609,23.988647,-8.187614,...,0.282580,0.253668,26.138117,26.106901,24.963594,25.121482,23.027536,24.736787,24.693225,gunshot
5,teste.wav,0.922455,3261.378912,2264.549261,0.211681,6130.657956,0.150714,216.474148,13.156833,-11.303448,...,0.257010,0.267818,22.742492,21.280518,21.403479,23.739221,25.055030,24.217401,24.365702,gunshot
6,teste.wav,0.915480,3423.811655,2238.859473,0.225848,6211.852543,0.162613,216.082409,10.366624,-10.610264,...,0.260141,0.259250,21.572463,23.782177,23.372112,22.828666,24.387265,23.792568,24.100986,gunshot
7,teste.wav,0.913170,3538.142654,2215.888691,0.244049,6282.781148,0.166463,216.161194,11.958274,-8.087863,...,0.265994,0.274961,22.336204,26.209764,20.729363,23.013760,24.697374,23.784630,24.763205,gunshot
8,teste.wav,0.916643,3543.576511,2224.956393,0.229570,6273.448437,0.159755,216.128716,16.630360,-3.158215,...,0.282659,0.274164,23.293726,23.894012,22.565578,26.502834,26.389561,24.415697,24.102551,gunshot
9,teste.wav,0.923279,3302.328013,2294.824833,0.200715,6195.986934,0.145174,216.608818,15.912603,-4.567915,...,0.301987,0.256291,27.257390,25.413653,24.098846,25.393067,25.595660,23.635867,23.658042,gunshot


#### Função para verificar qual é a classificação correta de acordo com o nome do arquivo

Essa é bem básica. Em algum determinado momento eu vou precisar saber a classficação correta de um determinado áudio. Eu só consigo saber isso pelo nome do arquivo. LEMBRANDO QUE TODO O CÓDIGO ESCRITO AQUI SERVE PARA O BANCO DE DADOS SESA. Os arquivos são nomeados como **classe_contador.wav**. Por exemplo: casual_000.wav, explosion_032.wav e gunshot_032.wav.

In [45]:
def verificarClassificacaoCorreta(nomeArquivo):
    arrayNome = nomeArquivo.split("_")
    return arrayNome[0]

In [46]:
diretorio = "/home/dimi/Downloads/SESA/test/"
parar = 10
for nomeArquivo in os.listdir(diretorio):
    print(verificarClassificacaoCorreta(nomeArquivo))
    parar -= 1
    if parar == 0:
        break

explosion
gunshot
explosion
gunshot
gunshot
casual
explosion
casual
casual
gunshot


#### Função que passa por todos os áudios da pasta e vai montando o dataframe

Essa é função que une todas as outras. Ela vai passar por todos os áudios da pasta, verificar qual é a classificação correta de cada áudio, extrair as features, gerar o dataframe de features do áudio atual e agregar ao dataframe de todos os áudios.

In [68]:
def montarDataframeTodosOsAudios(diretorio, freqAmostragem, frameLength, overlapLength):
    
    print("Fora da classe")
    
    # CRIANDO O ARRAY COM O NOME DOS ARQUIVOS
    arrayNomeArquivos = os.listdir(diretorio)
    
    # PEGANDO O TOTAL DE ARQUIVOS NA PASTA APENAS PARA FINS DE PRINT
    totalArquivosNaPasta = len(arrayNomeArquivos)
    
    # ABAIXO, VOU CRIAR O DATAFRAME DE TODOS OS AUDIOS
    dataframeGeral = pd.DataFrame()

    # VOU PASSAR POR TODOS OS AUDIOS DO DIRETORIO
    for i, nomeArquivo in enumerate(arrayNomeArquivos):
        # PRINTANDO O PROGRESSO
        print("Extraindo features do arquivo", i+1, "de", totalArquivosNaPasta, "-> " + str(100*((i+1)/totalArquivosNaPasta)) + "%")
        
        # ABRO O AUDIO ATUAL COM O LIBROSA
        audioAtual, freqAmostragem = librosa.load(audioTesteDir, sr=freqAmostragem, mono=True) 
        
        # VERIFICAO QUAL E A CLASSIFICACAO CORRETA
        classeAudioAtual = verificarClassificacaoCorreta(nomeArquivo)
        
        # MONTO A MATRIZ DE FEATUREA DE CADA FRAME DO AUDIO ATUAL (a funcao abaixo
        # devolve uma matriz normal e faz o janelamento em outra funcao tb)
        matrizFeaturesAudioAtual = extrairFeaturesUnicoAudio(audioAtual, freqAmostragem, frameLength, overlapLength)
        
        # AGORA E HORA DE COLOCAR O NOME E A CLASSIFICACAO CORRETA NA MATRIZ
        # MAAAS, NA FUNCAO ABAIXO, O BICHO VIRA UM DATAFRAME PANDAS
        dataframeAudioAtual      = adicionarNomeArquivoEClasse(matrizFeaturesAudioAtual, nomeArquivo, classeAudioAtual)
        
        # CONCATENANDO O DATAFRAME DO AUDIO ATUAL AO DATAFRAME GERAL
        dataframeGeral = pd.concat([dataframeGeral, dataframeAudioAtual])
        
    # POR ULTIMO, RETORNO O DATAFRAME GERAL
    return dataframeGeral

#### Função para escalonar as features

Essa função recebe o dataframe geral, remove as primeira e última colunas (nome e classificação), faz o escalonamento e depois coloca as colunas de nome e classificação de volta.

In [48]:
def escalonarFeatures(dataframeGeral):
    
    print("Escalonando features")
    
    # COPIANDO AS COLUNAS DE NOME E CLASSIFICACAO
    colunaArquivo       = dataframeGeral["nomeArquivo"]
    colunaClassificacao = dataframeGeral["classificacaoCorreta"]
    
    # DELETANDO AS COLUNAS ARQUIVO E CLASSIFICACAO
    dataframeGeral = dataframeGeral.drop(['nomeArquivo', 'classificacaoCorreta'], axis=1)
    
    # ESCALONANDO
    dataframeGeral = pd.DataFrame(StandardScaler().fit_transform(dataframeGeral))
    
    # COM ESSA COISA DE TIRA E POE COLUNA, O PANDAS NAO SABE LINDAR COM OS INDEXES,
    # ABAIXO EU ESTOU RESETANDO TUDO
    colunaArquivo.reset_index(inplace=True, drop=True)
    colunaClassificacao.reset_index(inplace=True, drop=True)
    dataframeGeral.reset_index(inplace=True, drop=True)
    
    # ADICIONANDO AS COLUNAS QUE FORAM EXCLUIDAS (posicaoNovaColuna, nomeNovaColuna, valorParaTodasAsLinhas)
    dataframeGeral.insert(0, "nomeArquivo", colunaArquivo, True)
    dataframeGeral.insert(len(dataframeGeral.columns), "classificacaoCorreta", colunaClassificacao, True)
    
    return dataframeGeral

#### Função que reduz a dimensionalidade do dataframe geral

Depois de escalonar, vamos reduzir a dimensionalidade utilizando PCA. A ideia é fazer com que o CSV já saia pronto para ser usado, logo, a redução de dimensionalidade deve vir nessa classe. Apesar disso, vou deixar o parâmetro **nDimensoes** lá no construtor da classe, ai caso ele seja nulo eu pulo a parte da redução de dimensionalidade e gero o CSV com todas as features.

Poder usar a classe sem fazer a redução de dimensionalidade vai ser muito importante para verificar qual é o melhor número de componentes principais do PCA, como se mostra https://towardsdatascience.com/an-approach-to-choosing-the-number-of-components-in-a-principal-component-analysis-pca-3b9f3d6e73fe

Então a ideia é primeiro gerar o CSV completo dos dados de treino, sem reduzir a dimensionalidade, pra depois fazer um estudo sobre o melhor número de componentes principais. Aí sim, com **nDimensoes** definido, eu começo a usar a classe com essa função.

In [49]:
def reduzirDimensionalidade(dataframeGeral, nDimensoes):
    
    print("Reduzindo a dimensionalidade")
    
    # COPIANDO AS COLUNAS DE NOME E CLASSIFICACAO
    colunaArquivo       = dataframeGeral["nomeArquivo"]
    colunaClassificacao = dataframeGeral["classificacaoCorreta"]
    
    # DELETANDO AS COLUNAS ARQUIVO E CLASSIFICACAO
    dataframeGeral = dataframeGeral.drop(['nomeArquivo', 'classificacaoCorreta'], axis=1)
    
    # REDUZINDO A DIMENSIONALIDADE
    dataframeGeral = pd.DataFrame(PCA(n_components=nDimensoes).fit_transform(dataframeGeral))
    
    # COM ESSA COISA DE TIRA E POE COLUNA, O PANDAS NAO SABE LINDAR COM OS INDEXES,
    # ABAIXO EU ESTOU RESETANDO TUDO
    colunaArquivo.reset_index(inplace=True, drop=True)
    colunaClassificacao.reset_index(inplace=True, drop=True)
    dataframeGeral.reset_index(inplace=True, drop=True)
    
    # ADICIONANDO AS COLUNAS QUE FORAM EXCLUIDAS (posicaoNovaColuna, nomeNovaColuna, valorParaTodasAsLinhas)
    dataframeGeral.insert(0, "nomeArquivo", colunaArquivo, True)
    dataframeGeral.insert(len(dataframeGeral.columns), "classificacaoCorreta", colunaClassificacao, True)
    
    return dataframeGeral

### Finalizando

Abaixo, vou criar a função que será a construtora da classe (init). Ela vai usar todas as funções que já foram criadas. Depois disso, vai bastar copiar todas as funções pra dentro da classe e fazer acontecer.

In [50]:
def construtor(diretorio, freqAmostragem, frameLength, overlapLength, nDimensoes=None):
    
    # PRIMEIRO EU CRIO O DATAFRAME DE TODOS OS AUDIOS DA PASTA
    dataframeGeral = montarDataframeTodosOsAudios(diretorio, freqAmostragem, frameLength, overlapLength)
    
    # DEPOIS EU ESCALONO AS FEATURES
    dataframeGeral = escalonarFeatures(dataframeGeral)
    
    # E AGORA EU REDUZO A DIMENSIONALIDADE
    if nDimensoes != None:
        dataframeGeral = reduzirDimensionalidade(dataframeGeral, nDimensoes)
        
    # PARA FINALIZAR, VOU ESCREVER O DATAFRAME NUM CSV
    nomeCSV = diretorio + str(time.time()) + ".csv"
    print("Escrevendo CSV:", nomeCSV)
    dataframeGeral.to_csv(nomeCSV, index=False)
    
    # TAMBEM VOU RETORNAR O DATAFRAME, PQ VAI QUE EU PRECISO NE
    print("Operação finalizada")
    return dataframeGeral

# FINALMENTE COLOCANDO TUDO DENTRO DA CLASSE

A primeira parte vai ter o construtor da classe. A segunda parte vai ser das funções intermediárias que fazem tudo acontecer, e só por último vou colocar as funções de extração de features.

In [73]:
# TODA A IMPLEMETACAO DESSA CLASSE ESTA MUITO BEM DOCUMENTADA NO JUPYTER "
# Implementação de classe para extração de features - SESA Dataset"
# VOU COLOCAR O MAXIMO DE COMENTARIOS POSSIVEIS AQUI TAMBEM, MAS PARA MAIS INFORMACOES
# E MELHOR OLHAR POR LA

# COMO USAR A CLASSE:
# A CLASSE ENTRA EM UMA DETERMINADA PASTA CONTENDO APENAS ARQUIVOS WAV E CRIA UM CSV DE FEATURES
# A CLASSE TAMBEM E RESPONSAVEL POR ESCALONAR AS FEATURES E FAZER UMA REDUCAO DE DIMENSIONALIDADE

# PRIMEIRO, OS PARAMETROS ABAIXOS DEVEM SER SETADOS:
# diretorio      -> PASTA ONDE A CLASSE VAI PROCURAR PELOS WAVs PARA GERAR O CSV
# freqAmostragem -> A FREQUENCIA DE AMOSTRAGEM DOS AUDIOS DESSA PASTA
# frameLength    -> O TAMANHO DAS JANELAS DE CADA AUDIO EM QTD DE FRAMES
# overlapLength  -> TAMANHO DA SOBREPOSICAO EM QTD DE FRAMES
# nDimensoes     -> QTD DE DIMENSOES UTILIZADAS NO PCA. CASO SEJA NULL, NAO HAVERA REDUCAO DE DIMENSIONALIDADE

# O COMANDO ABAIXO INSTANCIA A CLASSE. O CONSTRUTOR DEVOLVE UM DATAFRAME PANDAS AO MESMO TEMPO QUE CRIA O CSV
# dataframeGeral = extrairFeatures(diretorio, freqAmostragem, frameLength, overlapLength, nDimensoes)

import os
import librosa
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

class ExtrairFeatures:

	# CONSTRUTOR---------------------------------------------------------------------------------------------
	def __init__(self, diretorio, freqAmostragem, frameLength, overlapLength, escalonamento=True, nDimensoes=None):    
		# PRIMEIRO EU CRIO O DATAFRAME DE TODOS OS AUDIOS DA PASTA
		dataframeGeral = self.montarDataframeTodosOsAudios(diretorio, freqAmostragem, frameLength, overlapLength)
		
		# DEPOIS EU ESCALONO AS FEATURES
		if escalonamento == True:
			dataframeGeral = self.escalonarFeatures(dataframeGeral)
		
		# E AGORA EU REDUZO A DIMENSIONALIDADE
		if nDimensoes != None:
			dataframeGeral = self.reduzirDimensionalidade(dataframeGeral, nDimensoes)
			
		# PARA FINALIZAR, VOU ESCREVER O DATAFRAME NUM CSV
		nomeCSV = diretorio + str(time.time()) + ".csv"
		print("Escrevendo CSV:", nomeCSV)
		dataframeGeral.to_csv(nomeCSV, index=False)
		
		# TAMBEM VOU RETORNAR O DATAFRAME, PQ VAI QUE EU PRECISO NE
		print("Operação finalizada")
		
		# return dataframeGeral

	# DEFINICAO DE FUNCOES INTERMEDIARIAS--------------------------------------------------------------------	
	def fazerJanelamento(self, sinal, frameLength, overlapLength):
		# Função que faz o janelamento
		# Existe um problema em deixar que as funções de extração de features criadas acima façam o janelamento: 
		# ao invés de retornarem valores unitários para as features, elas vão retornar um array em que cada posição 
		# representa um janelamento. Portanto, a solução é fazer o janelamento antes de extrair as features e deixar 
		# para mandar para essas funções apenas as janelas, fazendo com que frameLength seja igual ao tamanho da 
		# janela que está sendo enviada e que overlapLength seja 0.
		# A função abaixo usa a função frame do librosa que retorna as janelas como COLUNAS. 
		# Como eu quero que cada janela seja uma LINHA, eu retorno a transposta dessa função.
		return librosa.util.frame(sinal, frame_length=frameLength, hop_length=overlapLength).T

	def extrairFeaturesUnicoFrame(self, sinal, freqAmostragem, frameLength):
		# Função que extrai features de um único frame
		# Em algum momento, a classe deverá passar em cada um dos áudios da pasta para ir extraindo as features. 
		# A próxima função extrai as features de um único frame de áudio e retorna um array com essas features. 
		# Posteriormente esse array deverá ser integrado à matriz de features de todos os áudios.
		# Haverá uma outra função para extrair as features de um único áudio. Ela deverá pegar um áudio, usar a 
		# função de janelamento, e usar a função abaixo para extrair as features de cada uma das janelas. Depois, 
		# ela vai retornar uma matriz com as features de cada frame de um único áudio.
		# 
		# PARA IMPEDIR QUE O LIBROSA CONTINUE FAZENDO O JANELAMENTO DO ÁUDIO MESMO QUE frameLength 
		# SEJA DO TAMANHO DO ÁUDIO, O PARÂMETRO DE OVERLAP DEVE SER MAIOR OU IGUAL A frameLength.s
		overlapLength = frameLength
		
		# CRIANDO O ARRAY DE FEATURES DO FRAME EM QUESTAO
		arrayFeaturesFrame = []
		
		#PRIMEIRO, VOU EXTRAIR AS FEATURES UNITARIAS
		arrayFeaturesFrame.append(float(self.extrairRMS(sinal, frameLength, overlapLength)))
		arrayFeaturesFrame.append(float(self.extrairCentroideEspectral(sinal, freqAmostragem, frameLength, overlapLength)))
		arrayFeaturesFrame.append(float(self.extrairLarguraBanda(sinal, freqAmostragem, frameLength, overlapLength)))
		arrayFeaturesFrame.append(float(self.extrairPlanicidade(sinal, frameLength, overlapLength)))
		arrayFeaturesFrame.append(float(self.extrairRolloff(sinal, freqAmostragem, frameLength, overlapLength)))
		arrayFeaturesFrame.append(float(self.extrairZCR(sinal, frameLength, overlapLength)))
		
		# AGORA VAMOS PASSAR PARA AS NAO UNITARIAS, PRIMEIRO, E PRECISO CRIAR A MATRIZ DOS MFCCS
		matrizMFCC          = self.extrairMatrizMFCC(sinal, freqAmostragem)
		
		# AGORA SIM EU SAIO EXTRAINDO AS FEATURES 
		arrayFeaturesFrame += self.extrairMFCCs(matrizMFCC)
		arrayFeaturesFrame += self.extrairDeltas(matrizMFCC)
		arrayFeaturesFrame += self.extrairDeltaDeltas(matrizMFCC)
		arrayFeaturesFrame += self.extrairMelEspectrograma(sinal, freqAmostragem, frameLength, overlapLength)
		arrayFeaturesFrame += self.extrairCromagramas(sinal, freqAmostragem, frameLength, overlapLength)
		arrayFeaturesFrame += self.extrairCromagramasQ(sinal, freqAmostragem)
		arrayFeaturesFrame += self.extrairCromaCENSs(sinal, freqAmostragem)
		arrayFeaturesFrame += self.extrairContrastes(sinal, freqAmostragem, frameLength, overlapLength)
		
		# POR FIM, RETORNO O ARRAY DE FEATURES DO AUDIO QUE FOI ENVIADO PARA ESSA FUNCAO
		return arrayFeaturesFrame

	def extrairFeaturesUnicoAudio(self, sinal, freqAmostragem, frameLength, overlapLength):
		# Função que extrai as features de um único áudio
		# Essa função vai pegar um áudio, usar a função de janelamento, e para cada janela do áudio em questão, 
		# ela vai usar a função de extrair as features de uma única janela (implementada acima). 
		# Depois, ela vai retornar uma matriz com as features de cada frame de um único áudio, onde cada linha 
		# é uma frame e cada coluna é uma feature.

		# PRIMEIRO, VOU CRIAR A MATRIZ QUE VAI CONTER AS FEATURES DE CADA JANELA
		# CADA LINHA E UMA JANELA E CADA COLUNA E UMA FEATURE
		matrizFeaturesAudio = []
		
		# DEPOIS, VOU FAZER O JANELAMENTO
		matrizFramesAudio = self.fazerJanelamento(sinal, frameLength, overlapLength)
		
		# AGORA, PARA CADA JANELA, VOU EXTRAIR AS FEATURES E COLOCAR COMO UMA LINHA NOVA NA MATRIZ
		for frameAtual in matrizFramesAudio:
			matrizFeaturesAudio.append(self.extrairFeaturesUnicoFrame(frameAtual, freqAmostragem, frameLength))
		
		# RETORNO A MATRIZ DE FEATURES DESSE AUDIO
		return matrizFeaturesAudio

	def adicionarNomeArquivoEClasse(self, matrizFeaturesAudioAtual, nomeArquivo, classificacaoCorreta):
		#Função que coloca o nome do arquivo e a classificação correta na matriz de features de um áudio
		#Haverá uma matriz de dados que terá as seguintes colunas: nomeArquivo, ...features... e classificacaoCorreta. 
		#Mas, a função implementada para gerar a matriz de features de um único áudio extrairFeaturesUnicoAudio 
		#devolve uma matriz de features sem o nome do áudio e a classificação correta. Portanto, a função abaixo 
		# apenas pega essa última matriz e coloca o nome do arquivo no começo e a classificação correta ao final. 
		# Posteriormente, o resultado será agregado à matriz de dados citada em primeiro lugar.

		#ESSA FUNÇÃO RETORNA UM DATAFRAME PANDAS, NÃO UMA MATRIZ QUALQUER.

		#https://www.geeksforgeeks.org/python-pandas-dataframe-insert/

		# PRIMEIRO TRANSFORMO A MATRIZ NUM PANDAS DATAFRAME
		dataframeAudioAtual = pd.DataFrame(matrizFeaturesAudioAtual)
		
		# AGORA COLOCO A COLUNA DO NOME NO COMECO (posicaoNovaColuna, nomeNovaColuna, valorParaTodasAsLinhas)
		dataframeAudioAtual.insert(0, "nomeArquivo", nomeArquivo, True)
		
		# AGORA COLOCO A COLUNA DA CLASSIFICACAO CORRETA NA ULTIMA POSICAO
		dataframeAudioAtual.insert(len(dataframeAudioAtual.columns), "classificacaoCorreta", classificacaoCorreta, True)
		
		return dataframeAudioAtual

	def verificarClassificacaoCorreta(self, nomeArquivo):
		#Função para verificar qual é a classificação correta de acordo com o nome do arquivo
		#Essa é bem básica. Em algum determinado momento eu vou precisar saber a classficação 
		# correta de um determinado áudio. Eu só consigo saber isso pelo nome do arquivo. LEMBRANDO QUE 
		# TODO O CÓDIGO ESCRITO AQUI SERVE PARA O BANCO DE DADOS SESA. Os arquivos são nomeados 
		# como classe_contador.wav. Por exemplo: casual_000.wav, explosion_032.wav e gunshot_032.wav.
		arrayNome = nomeArquivo.split("_")
		return arrayNome[0]

	def montarDataframeTodosOsAudios(self, diretorio, freqAmostragem, frameLength, overlapLength):
		# Função que passa por todos os áudios da pasta e vai montando o dataframe
		# Essa é função que une todas as outras. Ela vai passar por todos os áudios da pasta, 
		# verificar qual é a classificação correta de cada áudio, extrair as features, gerar o 
		# dataframe de features do áudio atual e agregar ao dataframe de todos os áudios.
	
		# CRIANDO O ARRAY COM O NOME DOS ARQUIVOS
		arrayNomeArquivos = os.listdir(diretorio)
		
		# PEGANDO O TOTAL DE ARQUIVOS NA PASTA APENAS PARA FINS DE PRINT
		totalArquivosNaPasta = len(arrayNomeArquivos)
		
		# ABAIXO, VOU CRIAR O DATAFRAME DE TODOS OS AUDIOS
		dataframeGeral = pd.DataFrame()

		# VOU PASSAR POR TODOS OS AUDIOS DO DIRETORIO
		for i, nomeArquivo in enumerate(arrayNomeArquivos):
			# PRINTANDO O PROGRESSO
			print("Extraindo features do arquivo", i+1, "de", totalArquivosNaPasta, "-> " + str(100*((i+1)/totalArquivosNaPasta)) + "%")
			
			# ABRO O AUDIO ATUAL COM O LIBROSA
			audioAtual, freqAmostragem = librosa.load(diretorio+nomeArquivo, sr=freqAmostragem, mono=True) 
			
			# VERIFICAO QUAL E A CLASSIFICACAO CORRETA
			classeAudioAtual = self.verificarClassificacaoCorreta(nomeArquivo)
			
			# MONTO A MATRIZ DE FEATUREA DE CADA FRAME DO AUDIO ATUAL (a funcao abaixo
			# devolve uma matriz normal e faz o janelamento em outra funcao tb)
			matrizFeaturesAudioAtual = self.extrairFeaturesUnicoAudio(audioAtual, freqAmostragem, frameLength, overlapLength)
			
			# AGORA E HORA DE COLOCAR O NOME E A CLASSIFICACAO CORRETA NA MATRIZ
			# MAAAS, NA FUNCAO ABAIXO, O BICHO VIRA UM DATAFRAME PANDAS
			dataframeAudioAtual      = self.adicionarNomeArquivoEClasse(matrizFeaturesAudioAtual, nomeArquivo, classeAudioAtual)
			
			# CONCATENANDO O DATAFRAME DO AUDIO ATUAL AO DATAFRAME GERAL
			dataframeGeral = pd.concat([dataframeGeral, dataframeAudioAtual])
			
		# POR ULTIMO, RETORNO O DATAFRAME GERAL
		return dataframeGeral

	def escalonarFeatures(self, dataframeGeral):
		
		# Função para escalonar as features
		# Essa função recebe o dataframe geral, remove as primeira e última colunas (nome e classificação), 
		# faz o escalonamento e depois coloca as colunas de nome e classificação de volta.
	
		print("Escalonando features")
		
		# COPIANDO AS COLUNAS DE NOME E CLASSIFICACAO
		colunaArquivo       = dataframeGeral["nomeArquivo"]
		colunaClassificacao = dataframeGeral["classificacaoCorreta"]
		
		# DELETANDO AS COLUNAS ARQUIVO E CLASSIFICACAO
		dataframeGeral = dataframeGeral.drop(['nomeArquivo', 'classificacaoCorreta'], axis=1)
		
		# ESCALONANDO
		dataframeGeral = pd.DataFrame(StandardScaler().fit_transform(dataframeGeral))
		
		# COM ESSA COISA DE TIRA E POE COLUNA, O PANDAS NAO SABE LINDAR COM OS INDEXES,
		# ABAIXO EU ESTOU RESETANDO TUDO
		colunaArquivo.reset_index(inplace=True, drop=True)
		colunaClassificacao.reset_index(inplace=True, drop=True)
		dataframeGeral.reset_index(inplace=True, drop=True)
		
		# ADICIONANDO AS COLUNAS QUE FORAM EXCLUIDAS (posicaoNovaColuna, nomeNovaColuna, valorParaTodasAsLinhas)
		dataframeGeral.insert(0, "nomeArquivo", colunaArquivo, True)
		dataframeGeral.insert(len(dataframeGeral.columns), "classificacaoCorreta", colunaClassificacao, True)
		
		return dataframeGeral

	def reduzirDimensionalidade(self, dataframeGeral, nDimensoes):
	
		# Função que reduz a dimensionalidade do dataframe geral
		# Depois de escalonar, vamos reduzir a dimensionalidade utilizando PCA. 
		# A ideia é fazer com que o CSV já saia pronto para ser usado, logo, a redução 
		# de dimensionalidade deve vir nessa classe. Apesar disso, vou deixar o parâmetro 
		# nDimensoes lá no construtor da classe, ai caso ele seja nulo eu pulo a parte da 
		# redução de dimensionalidade e gero o CSV com todas as features.

		print("Reduzindo a dimensionalidade")
		
		# COPIANDO AS COLUNAS DE NOME E CLASSIFICACAO
		colunaArquivo       = dataframeGeral["nomeArquivo"]
		colunaClassificacao = dataframeGeral["classificacaoCorreta"]
		
		# DELETANDO AS COLUNAS ARQUIVO E CLASSIFICACAO
		dataframeGeral = dataframeGeral.drop(['nomeArquivo', 'classificacaoCorreta'], axis=1)
		
		# REDUZINDO A DIMENSIONALIDADE
		dataframeGeral = pd.DataFrame(PCA(n_components=nDimensoes).fit_transform(dataframeGeral))
		
		# COM ESSA COISA DE TIRA E POE COLUNA, O PANDAS NAO SABE LINDAR COM OS INDEXES,
		# ABAIXO EU ESTOU RESETANDO TUDO
		colunaArquivo.reset_index(inplace=True, drop=True)
		colunaClassificacao.reset_index(inplace=True, drop=True)
		dataframeGeral.reset_index(inplace=True, drop=True)
		
		# ADICIONANDO AS COLUNAS QUE FORAM EXCLUIDAS (posicaoNovaColuna, nomeNovaColuna, valorParaTodasAsLinhas)
		dataframeGeral.insert(0, "nomeArquivo", colunaArquivo, True)
		dataframeGeral.insert(len(dataframeGeral.columns), "classificacaoCorreta", colunaClassificacao, True)
		
		return dataframeGeral

	# DEFINICAO DAS FUNCOES QUE REALMENTE EXTRAEM AS FEATURES -----------------------------------------------

	def extrairRMS(self, sinal, frameLength, overlapLength):
		return librosa.feature.rms(y=sinal, frame_length=frameLength, hop_length=overlapLength)

	def extrairCentroideEspectral(self, sinal, freqAmostragem, frameLength, overlapLength):
		return librosa.feature.spectral_centroid(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

	def extrairLarguraBanda(self, sinal, freqAmostragem, frameLength, overlapLength):
		return librosa.feature.spectral_bandwidth(y=sinal, sr= freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

	def extrairPlanicidade(self, sinal, frameLength, overlapLength):
		return librosa.feature.spectral_flatness(y=sinal, n_fft=frameLength, hop_length=overlapLength)

	def extrairRolloff(self, sinal, freqAmostragem, frameLength, overlapLength):
		return librosa.feature.spectral_rolloff(y=sinal, sr= freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

	def extrairZCR(self, sinal, frameLength, overlapLength):
		return librosa.feature.zero_crossing_rate(y=sinal, frame_length=frameLength, hop_length=overlapLength)

	def extrairMatrizMFCC(self, sinal, freqAmostragem):
		return librosa.feature.mfcc(y=sinal, sr=freqAmostragem)

	def extrairMFCCs(self, matrizMFCC):
		
		arrayMFCCs = []
		
		for linha in matrizMFCC:
			arrayMFCCs.append(np.mean(linha))
			
		return arrayMFCCs

	def extrairDeltas(self, matrizMFCC):
		matrizDelta = librosa.feature.delta(matrizMFCC, order=1)

		arrayDelta = []

		for linha in matrizDelta:
			arrayDelta.append(np.mean(linha))

		return arrayDelta

	def extrairDeltaDeltas(self, matrizMFCC):
		matrizDeltaDelta = librosa.feature.delta(matrizMFCC, order=2)

		arrayDeltaDelta = []

		for linha in matrizDeltaDelta:
			arrayDeltaDelta.append(np.mean(linha))

		return arrayDeltaDelta

	def extrairMelEspectrograma(self, sinal, freqAmostragem, frameLength, overlapLength):
		
		matrizMelEspectrograma = librosa.feature.melspectrogram(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)
		
		arrayMelEspectrograma = []
		
		for coluna in matrizMelEspectrograma.T:
			arrayMelEspectrograma.append(np.mean(coluna))
		
		return arrayMelEspectrograma

	def extrairCromagramas(self, sinal, freqAmostragem, frameLength, overlapLength):
		
		matrizCromagramas = librosa.feature.chroma_stft(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)
		
		arrayCromagramas = []
		
		for linha in matrizCromagramas:
			arrayCromagramas.append(np.mean(linha))
		
		return arrayCromagramas

	def extrairCromagramasQ(self, sinal, freqAmostragem):
		
		matrizCromagramasQ = librosa.feature.chroma_cqt(y=sinal, sr=freqAmostragem)
		
		arrayCromagramasQ = []
		
		for linha in matrizCromagramasQ:
			arrayCromagramasQ.append(np.mean(linha))
		
		return arrayCromagramasQ

	def extrairCromaCENSs(self, sinal, freqAmostragem):
		
		matrizCromaCENSs = librosa.feature.chroma_cens(y=sinal, sr=freqAmostragem)
		
		arrayCromaCENSs = []
		
		for linha in matrizCromaCENSs:
			arrayCromaCENSs.append(np.mean(linha))
		
		return arrayCromaCENSs

	def extrairContrastes(self, sinal, freqAmostragem, frameLength, overlapLength):
		
		matrizContrastes = librosa.feature.spectral_contrast(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)
		
		arrayConstrastes = []
		
		for linha in matrizContrastes:
			arrayConstrastes.append(np.mean(linha))
		
		return arrayConstrastes